# Pose Extractor

This notebook extracts estimated pose using a pre-trained OpenPose[1,2] model. 

 1. Extract frames for a given video (fps)
 2. Collect pose keypoints for all frames

#### References:

[1] *OpenPose: Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields*, Cao et al.

[2] https://github.com/CMU-Perceptual-Computing-Lab/openpose

In [1]:
videos_dir = '/usr/local/data02/zahra/datasets/Tempuckey/videos'
frames_dir = '/usr/local/data02/zahra/datasets/Tempuckey/frames'
op_output_dir = '/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose'

root_dir = '/usr/local/data01/zahra/repos/VideoFeatExtratotor'

## 1. Extract frames from a given video (fps)

In [2]:
from utils.sys_utils import *
from utils.video_utils import *

In [3]:
completed_vids = get_subfolders_path(frames_dir)
completed_vids = set([v.split('/')[-1] for v in completed_vids])

In [4]:
videos = get_files_path(videos_dir)

# extract frames
for v in videos:
    vname = v.replace('/','.').split('.')[-2]
    
    if vname in completed_vids:
        print('{} was previously completed\n'.format(vname))
        continue
        
    f = '{}/{}'.format(frames_dir, vname)
    create_folder(f)
    
    vc = cv2.VideoCapture(v)
    count = get_video_frames(vc, f)
    print('stored {} frames at {}\n'.format(count, f))


video_FACEOFF_000329 was previously completed

video_FACEOFF_000619 was previously completed

video_FACEOFF_000078 was previously completed

video_FACEOFF_000394 was previously completed

video_FACEOFF_000626 was previously completed

video_FACEOFF_000143 was previously completed

video_FACEOFF_000183 was previously completed

video_FACEOFF_000423 was previously completed

video_FACEOFF_000492 was previously completed

video_FACEOFF_000606 was previously completed

video_FACEOFF_000255 was previously completed

video_FACEOFF_000431 was previously completed

video_FACEOFF_000933 was previously completed

video_FACEOFF_000722 was previously completed

video_FACEOFF_000605 was previously completed

video_FACEOFF_000397 was previously completed

video_FACEOFF_000176 was previously completed

video_FACEOFF_000971 was previously completed

video_FACEOFF_000160 was previously completed

video_FACEOFF_000495 was previously completed

video_FACEOFF_000563 was previously completed

video_FACEOFF

stored 484 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000151

stored 374 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000680

stored 350 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000852

stored 587 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000851

stored 416 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000654

stored 576 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000907

stored 464 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000948

stored 349 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000287

stored 534 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000571

stored 518 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000125

stored 417 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames

stored 636 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000003

stored 522 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000061

stored 379 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000366

stored 700 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000442

stored 389 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000308

stored 530 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000210

stored 503 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000968

stored 350 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000015

stored 384 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000127

stored 778 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000284

stored 470 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames

stored 1094 frames at /usr/local/data02/zahra/datasets/Tempuckey/frames/video_FACEOFF_000063



## 2. Collect keypoints for all the frames in a clip 
### repeats for all clips and store heatmaps, keypoints, and ouput imgs for all frames in each clip

- ### <span style="color:green"> **2.1**</span> obtain joint heatmaps 
  - returns $H_j^t[x,y]$
  - in our BODY\_25 model, each image 1,2,...,25 contains all the same joints detected for everyone in the image

In [22]:
# reference: openpose python api
from openpose import pyopenpose as op
# setup the open pose model
params = dict()
params["model_folder"] = root_dir + "/models/"
params["heatmaps_add_parts"] = True
params["heatmaps_add_bkg"] = False
params["heatmaps_add_PAFs"] = False

# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

datum = op.Datum()

In [24]:
def store_op_output(poseHeatMaps_t, outputImages_t, keypoints_t, beg_frame_idx, end_frame_idx, vname, op_output_dir):
    
    beg_frame_idx = str(beg_frame_idx).zfill(5)
    end_frame_idx = str(end_frame_idx).zfill(5)
    
    # num_frames, num_keypoints/num_joints , img_width, img_height => dims = (T, 25, w, h)
    # i.e. for image frame 0 we have 25 heatmaps (one per keypoint or joint)
    output_path = '{}/{}_heatmaps_T_25_w_h_{}_{}'.format(op_output_dir, vname, beg_frame_idx, end_frame_idx)

    print('{}'.format(output_path))
    np.save(output_path , np.array(poseHeatMaps_t))

    # outputImages_t[0].shape => (720, 1280, 3) 
    # one RGB image with dims = (W,H,3) per frame. and a total of T frames
    output_path = '{}/{}_posed_imgs_T_w_h_3_{}_{}'.format(op_output_dir, vname, beg_frame_idx, end_frame_idx)

    print('{}'.format(output_path))
    np.save(output_path , np.array(outputImages_t))

    # keypoints_t dims = (T, num_persons, num_keypoints, 3) => where 3 is [x,y,confidence] for each detected keypoint on a person
    output_path = '{}/{}_keypoints_T_persons_25_3_{}_{}'.format(op_output_dir, vname, beg_frame_idx, end_frame_idx)

    print('{}'.format(output_path))
    np.save(output_path , np.array(keypoints_t))
    print()


In [25]:
labels_dir = '/usr/local/data02/zahra/datasets/Tempuckey/labels/ground_truth/tempuckey_ground_truth_labels.csv'
labels = read_csv(labels_dir)
faceoff_vids = set([v[1].split('.')[0] for v in labels if 'mp4' in v[1]])
# print(list(faceoff_vids)[:3])
# print(len(faceoff_vids))

In [26]:
completed_vids = get_files_path(op_output_dir)
completed_vids = set(['_'.join(v.split('/')[-1].split('_')[:3]) for v in completed_vids])
# print(list(completed_vids)[:2])
# print(len(completed_vids))

In [ ]:
videos = get_files_path(videos_dir)
import numpy as np

win = 320 # pack 320 frames into one npy file 

# extract pose heatmap and keypoints for each frame
for v in videos:
    vname = v.replace('/','.').split('.')[-2]
    
    if vname in completed_vids:
        print('{} was previously completed'.format(vname))
        continue
        
    if vname in faceoff_vids:
        print('{} is not a face-off vid. skipping.'.format(vname))
        continue
        
    f = '{}/{}'.format(frames_dir, vname)
    frames = get_files_path(f)
    
    poseHeatMaps_t = []
    outputImages_t = []
    keypoints_t = []

    # framewise pose, heatmap, keypoint extraction
    for i in range(len(frames)):
        img = cv2.imread(frames[i])

        datum.cvInputData = img
        opWrapper.emplaceAndPop([datum])

        poseHeatMaps_t.append(datum.poseHeatMaps.copy())
        outputImages_t.append(datum.cvOutputData.copy())
        keypoints_t.append(datum.poseKeypoints.copy())
        
        if (i+1)%win == 0:
            
            beg_frame_idx = i - (win-1)
            end_frame_idx = i

            store_op_output(poseHeatMaps_t, outputImages_t, keypoints_t, beg_frame_idx, end_frame_idx, vname, op_output_dir)
            
            poseHeatMaps_t = []
            outputImages_t = []
            keypoints_t = []
            
    if i > end_frame_idx:
        
        beg_frame_idx = end_frame_idx
        end_frame_idx = i

        store_op_output(poseHeatMaps_t, outputImages_t, keypoints_t, beg_frame_idx, end_frame_idx, vname, op_output_dir)


/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000329_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000329_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000329_keypoints_T_persons_25_3_00000_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000329_heatmaps_T_25_w_h_00319_00471
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000329_posed_imgs_T_w_h_3_00319_00471
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000329_keypoints_T_persons_25_3_00319_00471

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000619_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000619_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000619_keypoints_T_persons_25_3_00000

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000143_keypoints_T_persons_25_3_00319_00492

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000183_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000183_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000183_keypoints_T_persons_25_3_00000_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000183_heatmaps_T_25_w_h_00319_00390
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000183_posed_imgs_T_w_h_3_00319_00390
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000183_keypoints_T_persons_25_3_00319_00390

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000423_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000423_posed_imgs_T_w_h_3_0000

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000397_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000397_keypoints_T_persons_25_3_00000_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000397_heatmaps_T_25_w_h_00319_00554
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000397_posed_imgs_T_w_h_3_00319_00554
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000397_keypoints_T_persons_25_3_00319_00554

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000176_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000176_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000176_keypoints_T_persons_25_3_00000_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000176_heatmaps_T_25_w_h_0032

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000195_heatmaps_T_25_w_h_00319_00455
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000195_posed_imgs_T_w_h_3_00319_00455
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000195_keypoints_T_persons_25_3_00319_00455

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000803_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000803_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000803_keypoints_T_persons_25_3_00000_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000803_heatmaps_T_25_w_h_00319_00581
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000803_posed_imgs_T_w_h_3_00319_00581
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000803_keypoints_T_persons_25_3_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000337_heatmaps_T_25_w_h_00319_00551
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000337_posed_imgs_T_w_h_3_00319_00551
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000337_keypoints_T_persons_25_3_00319_00551

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000147_heatmaps_T_25_w_h_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000147_posed_imgs_T_w_h_3_00000_00319
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000147_keypoints_T_persons_25_3_00000_00319

/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000147_heatmaps_T_25_w_h_00319_00553
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000147_posed_imgs_T_w_h_3_00319_00553
/usr/local/data02/zahra/datasets/Tempuckey/feats/openpose/video_FACEOFF_000147_keypoints_T_persons_25_3_00319

In [ ]:
print('datum.poseHeatMaps.shape: \t', datum.poseHeatMaps.shape) #25_body_joints

# heat maps storing format: body parts + background + PAFs
print('datum.poseKeypoints.shape: \t', datum.poseKeypoints.shape) # (num_persons, num_joints_25, (x,y,confidence) )

print('datum.cvInputData.shape: \t', datum.cvInputData.shape)